In [5]:
import pandas as pd
pd.set_option("display.max_columns", None) 

df = pd.read_csv("jl_fs/train.csv")
df["image_path"] = df["sample_id"].apply(lambda x : f"jl_fs/images/train/{x}.jpg")
df.to_csv("train_updated.csv", index = False)

In [6]:
import os
import math
import random
import json
from dataclasses import dataclass
from typing import Optional, List

import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from PIL import Image

from transformers import (
    CLIPModel,
    AutoProcessor,
    get_linear_schedule_with_warmup,
)

In [14]:

# --------------------------- Config ---------------------------
CSV_PATH        = os.environ.get("TRAIN_CSV", "train_updated.csv")     # must contain text + price + image path
TEXT_COL        = os.environ.get("TEXT_COL", "catalog_content")
PRICE_COL       = os.environ.get("PRICE_COL", "price")
IMG_COL         = os.environ.get("IMG_COL",  "image_path")             # local jpg path column
ID_COL          = os.environ.get("ID_COL",  "sample_id")

MODEL_ID        = os.environ.get("MODEL_ID", "openai/clip-vit-large-patch14")
OUTPUT_DIR      = os.environ.get("OUTPUT_DIR", "price_clip_fulltrain")

SEED            = int(os.environ.get("SEED", "42"))
MAX_LEN         = int(os.environ.get("MAX_LEN", "64"))                 # CLIP text context is shorter
BATCH_SIZE      = int(os.environ.get("BATCH_SIZE", "32"))
LR              = float(os.environ.get("LR", "2e-5"))
WEIGHT_DECAY    = float(os.environ.get("WEIGHT_DECAY", "0.01"))
EPOCHS          = int(os.environ.get("EPOCHS", "5"))
WARMUP_RATIO    = float(os.environ.get("WARMUP_RATIO", "0.06"))
GRAD_ACCUM      = int(os.environ.get("GRAD_ACCUM", "1"))
MAX_GRAD_NORM   = float(os.environ.get("MAX_GRAD_NORM", "1.0"))
FP16            = os.environ.get("FP16", "true").lower() == "true"

# Loss & regularization
ALPHA_CONTRAST  = float(os.environ.get("ALPHA_CONTRAST", "0.25"))      # weight for contrastive loss (0..1)
TAU             = float(os.environ.get("TAU", "0.07"))                 # InfoNCE temperature
HUBER_DELTA     = float(os.environ.get("HUBER_DELTA", "1.0"))

# Price/log transform
MIN_PRICE       = float(os.environ.get("MIN_PRICE", "1e-6"))

# Missing image policy for TRAIN: zero | text_only | drop
IMG_MISSING_POLICY = os.environ.get("IMG_MISSING_POLICY", "zero").lower()
assert IMG_MISSING_POLICY in {"zero", "text_only", "drop"}

# Inference/Test config (optional)
TEST_CSV        = os.environ.get("TEST_CSV", "jl_fs/test.csv").strip()               # if "", inference is skipped
TEST_IMG_DIR    = os.environ.get("TEST_IMG_DIR", "jl_fs/images/test")  # used if test CSV lacks image_path

os.makedirs(OUTPUT_DIR, exist_ok=True)

In [8]:

# --------------------------- Utils ---------------------------
def set_seed(seed: int = SEED):
    random.seed(seed); np.random.seed(seed)
    torch.manual_seed(seed); torch.cuda.manual_seed_all(seed)

set_seed(SEED)

def smape_np(y_true, y_pred, eps=1e-8):
    y_true = np.asarray(y_true, dtype=np.float64)
    y_pred = np.asarray(y_pred, dtype=np.float64)
    denom = (np.abs(y_true) + np.abs(y_pred) + eps) / 2.0
    return 100.0 * np.mean(np.abs(y_pred - y_true) / denom)

def log2_price(p: np.ndarray) -> np.ndarray:
    return np.log2(np.clip(p, MIN_PRICE, None))

def delog2(x: np.ndarray) -> np.ndarray:
    return np.power(2.0, x)

# --------------------------- Dataset & Collate ---------------------------
class ClipPriceDataset(Dataset):
    """
    TRAIN:
      policy 'zero':      returns dummy pixel for missing images; later masked to zeros.
      policy 'text_only': returns no pixel_values for missing images; vision forward skipped.
      policy 'drop':      drops rows with missing images at dataset build time.

    TEST:
      We will ALWAYS behave like 'zero' (never drop predictions).

    Items may contain: input_ids, attention_mask, (pixel_values), img_missing, (target)
    """
    def __init__(self, df: pd.DataFrame, text_col: str, img_col: str,
                 prices_log2: Optional[np.ndarray],
                 processor: AutoProcessor, max_len: int, policy: str, is_test: bool = False):
        self.processor = processor
        self.max_len = max_len
        self.policy = policy
        self.is_test = is_test

        df = df.reset_index(drop=True).copy()
        df[text_col] = df[text_col].fillna("").astype(str)

        if (policy == "drop") and (not is_test):
            before = len(df)
            df = df[df[img_col].apply(lambda p: isinstance(p, str) and len(p) > 0 and os.path.exists(p))]
            self.dropped_missing = before - len(df)
        else:
            self.dropped_missing = 0

        self.texts = df[text_col].tolist()
        self.img_paths = df[img_col].fillna("").astype(str).tolist()
        self.prices_log2 = prices_log2
        self.missing_img_count = 0

        # Dummy pixel to get correct shape
        dummy = self.processor(images=Image.new("RGB", (224, 224)), return_tensors="pt")
        self._dummy_pixel = dummy["pixel_values"].squeeze(0)  # (C,H,W)

        # For exporting IDs in test predictions
        self.ids = df[ID_COL].tolist() if (ID_COL in df.columns) else list(range(len(df)))

    def __len__(self): return len(self.texts)

    def _load_image(self, path: str):
        if isinstance(path, str) and path and os.path.exists(path):
            try:
                return Image.open(path).convert("RGB")
            except Exception:
                pass
        self.missing_img_count += 1
        return None

    def __getitem__(self, idx):
        text = self.texts[idx]
        img  = self._load_image(self.img_paths[idx])

        enc_text = self.processor(text=[text], padding=False, truncation=True,
                                  max_length=self.max_len, return_tensors="pt")

        img_missing = 0
        pixel_values = None

        if img is None:
            img_missing = 1
            if self.is_test:
                # For test we NEVER drop; force zero-like behavior
                pixel_values = self._dummy_pixel.clone()
            else:
                if self.policy == "zero":
                    pixel_values = self._dummy_pixel.clone()
                elif self.policy == "text_only":
                    pixel_values = None
                elif self.policy == "drop":
                    # should not occur because drop was handled in __init__
                    pixel_values = self._dummy_pixel.clone()
        else:
            enc_img = self.processor(images=img, return_tensors="pt")
            pixel_values = enc_img["pixel_values"].squeeze(0)

        item = {
            "input_ids": enc_text["input_ids"].squeeze(0),
            "attention_mask": enc_text["attention_mask"].squeeze(0),
            "img_missing": torch.tensor(img_missing, dtype=torch.uint8),
            "row_id": torch.tensor(self.ids[idx], dtype=torch.long)
        }
        if pixel_values is not None:
            item["pixel_values"] = pixel_values
        if self.prices_log2 is not None:
            item["target"] = torch.tensor(self.prices_log2[idx], dtype=torch.float32)
        return item

In [9]:

@dataclass
class CollateClip:
    processor: AutoProcessor
    def __call__(self, batch):
        # pad text
        input_ids = [b["input_ids"] for b in batch]
        attention = [b["attention_mask"] for b in batch]
        text_padded = self.processor.tokenizer.pad(
            {"input_ids": input_ids, "attention_mask": attention},
            padding=True, return_tensors="pt"
        )
        # images: some may be absent (text_only policy)
        has_pix = [("pixel_values" in b) for b in batch]
        pixel_values = None
        if any(has_pix):
            shapes = [b["pixel_values"].shape for b in batch if "pixel_values" in b]
            C,H,W = shapes[0]
            stacked = []
            for b in batch:
                if "pixel_values" in b:
                    stacked.append(b["pixel_values"])
                else:
                    stacked.append(torch.zeros((C,H,W), dtype=torch.float32))
            pixel_values = torch.stack(stacked, dim=0)

        res = {
            "input_ids": text_padded["input_ids"],
            "attention_mask": text_padded["attention_mask"],
            "img_missing": torch.stack([b["img_missing"] for b in batch], dim=0),
            "row_id": torch.stack([b["row_id"] for b in batch], dim=0),
        }
        if pixel_values is not None:
            res["pixel_values"] = pixel_values
        if "target" in batch[0]:
            res["target"] = torch.stack([b["target"] for b in batch], dim=0)
        return res

# --------------------------- Model & Loss ---------------------------
class ClipRegressionHead(nn.Module):
    def __init__(self, embed_dim: int, dropout: float = 0.1):
        super().__init__()
        in_dim = 2 * embed_dim  # concat image+text
        self.net = nn.Sequential(
            nn.Dropout(dropout),
            nn.Linear(in_dim, in_dim),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(in_dim, 1),
        )
    def forward(self, x): return self.net(x).squeeze(-1)

def info_nce(z_img: torch.Tensor, z_txt: torch.Tensor, tau: float = 0.07) -> torch.Tensor:
    z_img = F.normalize(z_img, dim=-1)
    z_txt = F.normalize(z_txt, dim=-1)
    logits = torch.matmul(z_img, z_txt.t()) / tau  # (B,B)
    labels = torch.arange(z_img.size(0), device=z_img.device)
    loss_i = F.cross_entropy(logits, labels)
    loss_t = F.cross_entropy(logits.t(), labels)
    return 0.5 * (loss_i + loss_t)

def huber_loss(pred, target, delta=1.0):
    return F.huber_loss(pred, target, delta=delta)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)


In [10]:
# --------------------------- Load TRAIN data ---------------------------
print(f"🔧 Loading TRAIN CSV: {CSV_PATH}")
df = pd.read_csv(CSV_PATH)

# checks
for col, name in [(TEXT_COL, "TEXT_COL"), (PRICE_COL, "PRICE_COL"), (IMG_COL, "IMG_COL")]:
    if col not in df.columns:
        raise ValueError(f"{name} '{col}' not in CSV columns={df.columns.tolist()}")

# clean
df[TEXT_COL] = df[TEXT_COL].fillna("").astype(str).str.strip()
df = df.loc[pd.to_numeric(df[PRICE_COL], errors="coerce").notnull()].copy()
df[PRICE_COL] = df[PRICE_COL].astype(float)
df = df.loc[df[PRICE_COL] >= 0.0].reset_index(drop=True)

print(f"📦 Train rows: {len(df)}")

# targets (log2)
y_log = log2_price(df[PRICE_COL].values)

# --------------------------- CLIP backbone ---------------------------
processor = AutoProcessor.from_pretrained(MODEL_ID)
clip_model = CLIPModel.from_pretrained(MODEL_ID)

device = "cuda" if torch.cuda.is_available() else "cpu"
clip_model = clip_model.to(device)
price_head = ClipRegressionHead(embed_dim=clip_model.config.projection_dim, dropout=0.1).to(device)

print(f"🖥️ Device: {device}")
# Train end-to-end (set to False to freeze CLIP)
for p in clip_model.parameters():
    p.requires_grad = True

print(f"🧮 Trainable params CLIP={count_parameters(clip_model):,} | head={count_parameters(price_head):,}")

train_ds = ClipPriceDataset(df, TEXT_COL, IMG_COL, y_log, processor, MAX_LEN, IMG_MISSING_POLICY, is_test=False)
collate  = CollateClip(processor)
train_loader = DataLoader(train_ds, batch_size=BATCH_SIZE, shuffle=True,
                          num_workers=2, pin_memory=True, collate_fn=collate)


🔧 Loading TRAIN CSV: train_updated.csv
📦 Train rows: 75000


preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/905 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


...

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

Xet Storage is enabled for this repo, but the 'hf_xet' package is not installed. Falling back to regular HTTP download. For better performance, install the package with: `pip install huggingface_hub[hf_xet]` or `pip install hf_xet`


model.safetensors:   0%|          | 0.00/1.71G [00:00<?, ?B/s]

🖥️ Device: cuda
🧮 Trainable params CLIP=427,616,513 | head=2,362,369


In [11]:
no_decay = ["bias", "LayerNorm.weight"]
params = list(clip_model.named_parameters()) + [(f"head.{n}", p) for n, p in price_head.named_parameters()]
grouped = [
    {"params": [p for n, p in params if not any(nd in n for nd in no_decay)], "weight_decay": WEIGHT_DECAY},
    {"params": [p for n, p in params if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = torch.optim.AdamW(grouped, lr=LR)
num_training_steps = EPOCHS * max(1, math.ceil(len(train_loader) / max(1, GRAD_ACCUM)))
num_warmup = int(num_training_steps * WARMUP_RATIO)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=num_warmup, num_training_steps=num_training_steps
)
scaler = torch.cuda.amp.GradScaler(enabled=FP16)

# Warmup batch to tally missing images
print("🔎 Warmup batch to tally missing images…")
if len(train_loader) > 0:
    _ = next(iter(train_loader))
print(f"⚠️ TRAIN missing images: {train_ds.missing_img_count}")
print(f"🗑️ TRAIN dropped (policy={IMG_MISSING_POLICY}): {getattr(train_ds,'dropped_missing',0)}")

🔎 Warmup batch to tally missing images…


/tmp/ipykernel_97/2161992421.py:13: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = torch.cuda.amp.GradScaler(enabled=FP16)
You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


⚠️ TRAIN missing images: 0
🗑️ TRAIN dropped (policy=zero): 0


In [12]:
# --------------------------- TRAIN (Full data) ---------------------------
for epoch in range(1, EPOCHS + 1):
    clip_model.train(); price_head.train()
    loss_run = reg_run = con_run = 0.0

    optimizer.zero_grad(set_to_none=True)

    for step, batch in enumerate(train_loader, 1):
        input_ids      = batch["input_ids"].to(device, non_blocking=True)
        attention_mask = batch["attention_mask"].to(device, non_blocking=True)
        targets        = batch["target"].to(device, non_blocking=True).float()
        img_missing    = batch["img_missing"].to(device)

        with torch.cuda.amp.autocast(enabled=FP16):
            # Text features
            txt_feat = clip_model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)

            # Image features
            do_vision = ("pixel_values" in batch) and (IMG_MISSING_POLICY != "text_only")
            if do_vision:
                pixel_values = batch["pixel_values"].to(
                    device,
                    dtype=next(clip_model.vision_model.parameters()).dtype,
                    non_blocking=True
                )
                img_feat = clip_model.get_image_features(pixel_values=pixel_values)
                if img_missing.any():
                    img_feat = img_feat * (1.0 - img_missing.unsqueeze(1).float())
            else:
                img_feat = torch.zeros_like(txt_feat)

            # Normalize + fuse
            txt_n = F.normalize(txt_feat, dim=-1)
            img_n = F.normalize(img_feat, dim=-1)
            fused = torch.cat([img_n, txt_n], dim=-1)

            # Losses
            pred_log = price_head(fused)
            reg_loss = huber_loss(pred_log, targets, delta=HUBER_DELTA)

            con_loss = torch.tensor(0.0, device=device, dtype=txt_n.dtype)
            valid_idx = (img_missing == 0).nonzero(as_tuple=False).squeeze(-1)
            if do_vision and valid_idx.numel() > 1 and ALPHA_CONTRAST > 0:
                con_loss = info_nce(img_n[valid_idx], txt_n[valid_idx], tau=TAU)

            loss = (1.0 - ALPHA_CONTRAST) * reg_loss + ALPHA_CONTRAST * con_loss

        scaler.scale(loss).backward()

        if step % GRAD_ACCUM == 0:
            scaler.unscale_(optimizer)
            nn.utils.clip_grad_norm_(list(clip_model.parameters()) + list(price_head.parameters()), MAX_GRAD_NORM)
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
            scheduler.step()

        loss_run += float(loss.item())
        reg_run  += float(reg_loss.item())
        con_run  += float(con_loss.item()) if isinstance(con_loss, torch.Tensor) else float(con_loss)

        if step % 200 == 0:
            print(f"epoch {epoch} step {step}/{len(train_loader)} "
                  f"loss={loss_run/step:.4f} reg={reg_run/step:.4f} con={con_run/step:.4f}")

    print(f"✅ Epoch {epoch} done. avg_loss={loss_run/max(1,len(train_loader)):.4f}")

# Save final full-data checkpoint
full_ckpt = os.path.join(OUTPUT_DIR, "full_clip.pt")
torch.save(
    {
        "clip_state": clip_model.state_dict(),
        "head_state": price_head.state_dict(),
        "model_id": MODEL_ID,
        "config": {
            "ALPHA_CONTRAST": ALPHA_CONTRAST,
            "TAU": TAU,
            "MAX_LEN": MAX_LEN,
            "projection_dim": clip_model.config.projection_dim,
            "IMG_MISSING_POLICY": IMG_MISSING_POLICY,
            "HUBER_DELTA": HUBER_DELTA
        },
        "columns": {"id": ID_COL, "text": TEXT_COL, "image": IMG_COL, "price": PRICE_COL},
    },
    full_ckpt
)
print(f"💾 Saved full-data checkpoint to: {full_ckpt}")

with open(os.path.join(OUTPUT_DIR, "metrics_fulltrain.json"), "w") as f:
    json.dump({
        "train_rows": int(len(df)),
        "train_missing_images": int(train_ds.missing_img_count),
        "dropped_train": int(getattr(train_ds, "dropped_missing", 0)),
        "epochs": EPOCHS,
        "batch_size": BATCH_SIZE,
        "lr": LR,
        "weight_decay": WEIGHT_DECAY,
        "grad_accum": GRAD_ACCUM,
        "fp16": FP16,
    }, f, indent=2)

You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipykernel_97/864095044.py:14: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=FP16):


epoch 1 step 200/2344 loss=2.4677 reg=3.1764 con=0.3417
epoch 1 step 400/2344 loss=2.2198 reg=2.8369 con=0.3687
epoch 1 step 600/2344 loss=1.8211 reg=2.2815 con=0.4402
epoch 1 step 800/2344 loss=1.5155 reg=1.8749 con=0.4372
epoch 1 step 1000/2344 loss=1.3087 reg=1.6081 con=0.4105
epoch 1 step 1200/2344 loss=1.1635 reg=1.4236 con=0.3833
epoch 1 step 1400/2344 loss=1.0549 reg=1.2864 con=0.3604
epoch 1 step 1600/2344 loss=0.9722 reg=1.1834 con=0.3387
epoch 1 step 1800/2344 loss=0.9060 reg=1.1014 con=0.3198
epoch 1 step 2000/2344 loss=0.8526 reg=1.0357 con=0.3034
epoch 1 step 2200/2344 loss=0.8078 reg=0.9805 con=0.2898
✅ Epoch 1 done. avg_loss=0.7801


You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


epoch 2 step 200/2344 loss=0.2855 reg=0.3482 con=0.0975
epoch 2 step 400/2344 loss=0.2816 reg=0.3421 con=0.1000
epoch 2 step 600/2344 loss=0.2830 reg=0.3441 con=0.0997
epoch 2 step 800/2344 loss=0.2844 reg=0.3463 con=0.0988
epoch 2 step 1000/2344 loss=0.2840 reg=0.3462 con=0.0972
epoch 2 step 1200/2344 loss=0.2850 reg=0.3477 con=0.0968
epoch 2 step 1400/2344 loss=0.2834 reg=0.3459 con=0.0958
epoch 2 step 1600/2344 loss=0.2822 reg=0.3446 con=0.0950
epoch 2 step 1800/2344 loss=0.2821 reg=0.3445 con=0.0950
epoch 2 step 2000/2344 loss=0.2826 reg=0.3455 con=0.0939
epoch 2 step 2200/2344 loss=0.2815 reg=0.3442 con=0.0934
✅ Epoch 2 done. avg_loss=0.2803


You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


epoch 3 step 200/2344 loss=0.1727 reg=0.2113 con=0.0569
epoch 3 step 400/2344 loss=0.1767 reg=0.2160 con=0.0590
epoch 3 step 600/2344 loss=0.1774 reg=0.2167 con=0.0594
epoch 3 step 800/2344 loss=0.1759 reg=0.2151 con=0.0585
epoch 3 step 1000/2344 loss=0.1765 reg=0.2159 con=0.0580
epoch 3 step 1200/2344 loss=0.1770 reg=0.2167 con=0.0578
epoch 3 step 1400/2344 loss=0.1772 reg=0.2172 con=0.0574
epoch 3 step 1600/2344 loss=0.1771 reg=0.2171 con=0.0570
epoch 3 step 1800/2344 loss=0.1770 reg=0.2172 con=0.0565
epoch 3 step 2000/2344 loss=0.1760 reg=0.2161 con=0.0559
epoch 3 step 2200/2344 loss=0.1767 reg=0.2170 con=0.0556
✅ Epoch 3 done. avg_loss=0.1764


You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


epoch 4 step 200/2344 loss=0.0991 reg=0.1187 con=0.0404
epoch 4 step 400/2344 loss=0.1007 reg=0.1209 con=0.0402
epoch 4 step 600/2344 loss=0.0994 reg=0.1191 con=0.0402
epoch 4 step 800/2344 loss=0.1001 reg=0.1201 con=0.0402
epoch 4 step 1000/2344 loss=0.1000 reg=0.1200 con=0.0401
epoch 4 step 1200/2344 loss=0.0999 reg=0.1200 con=0.0396
epoch 4 step 1400/2344 loss=0.0996 reg=0.1197 con=0.0393
epoch 4 step 1600/2344 loss=0.0991 reg=0.1191 con=0.0390
epoch 4 step 1800/2344 loss=0.0988 reg=0.1188 con=0.0386
epoch 4 step 2000/2344 loss=0.0983 reg=0.1183 con=0.0383
epoch 4 step 2200/2344 loss=0.0981 reg=0.1182 con=0.0379
✅ Epoch 4 done. avg_loss=0.0979


You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


epoch 5 step 200/2344 loss=0.0553 reg=0.0639 con=0.0295
epoch 5 step 400/2344 loss=0.0563 reg=0.0653 con=0.0294
epoch 5 step 600/2344 loss=0.0562 reg=0.0652 con=0.0289
epoch 5 step 800/2344 loss=0.0549 reg=0.0635 con=0.0289
epoch 5 step 1000/2344 loss=0.0544 reg=0.0631 con=0.0284
epoch 5 step 1200/2344 loss=0.0541 reg=0.0627 con=0.0281
epoch 5 step 1400/2344 loss=0.0532 reg=0.0616 con=0.0278
epoch 5 step 1600/2344 loss=0.0529 reg=0.0613 con=0.0277
epoch 5 step 1800/2344 loss=0.0525 reg=0.0609 con=0.0274
epoch 5 step 2000/2344 loss=0.0523 reg=0.0607 con=0.0274
epoch 5 step 2200/2344 loss=0.0521 reg=0.0603 con=0.0272
✅ Epoch 5 done. avg_loss=0.0516
💾 Saved full-data checkpoint to: price_clip_fulltrain/full_clip.pt


In [15]:

# --------------------------- INFERENCE (TEST) ---------------------------
def build_test_df(path: str, id_col: str, text_col: str, img_col: str, fallback_img_dir: str) -> pd.DataFrame:
    dft = pd.read_csv(path)
    # ensure id col exists
    if id_col not in dft.columns:
        raise ValueError(f"ID_COL '{id_col}' not in TEST_CSV columns={dft.columns.tolist()}")

    # text column can be absent; if so, create empty
    if text_col not in dft.columns:
        dft[text_col] = ""

    # image path column: create if absent
    if img_col not in dft.columns:
        dft[img_col] = dft[id_col].astype(str).apply(lambda x: os.path.join(fallback_img_dir, f"{x}.jpg"))

    dft[text_col] = dft[text_col].fillna("").astype(str).str.strip()
    dft[img_col]  = dft[img_col].fillna("").astype(str)
    return dft

@torch.no_grad()
def infer_predictions(clip_model: CLIPModel, price_head: nn.Module,
                      processor: AutoProcessor, df_test: pd.DataFrame,
                      text_col: str, img_col: str, batch_size: int = 64) -> np.ndarray:
    clip_model.eval(); price_head.eval()
    ds = ClipPriceDataset(df_test, text_col, img_col, prices_log2=None,
                          processor=processor, max_len=MAX_LEN, policy="zero", is_test=True)
    collate = CollateClip(processor)
    dl = DataLoader(ds, batch_size=batch_size, shuffle=False, num_workers=2, pin_memory=True, collate_fn=collate)

    preds_log = []
    for batch in dl:
        input_ids      = batch["input_ids"].to(device, non_blocking=True)
        attention_mask = batch["attention_mask"].to(device, non_blocking=True)
        img_missing    = batch["img_missing"].to(device)

        # Text
        txt_feat = clip_model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)

        # Image (always present as zero-fallback in test)
        pixel_values = batch["pixel_values"].to(
            device,
            dtype=next(clip_model.vision_model.parameters()).dtype,
            non_blocking=True
        )
        img_feat = clip_model.get_image_features(pixel_values=pixel_values)
        if img_missing.any():
            img_feat = img_feat * (1.0 - img_missing.unsqueeze(1).float())

        txt_n = F.normalize(txt_feat, dim=-1)
        img_n = F.normalize(img_feat, dim=-1)
        fused = torch.cat([img_n, txt_n], dim=-1)
        pred  = price_head(fused)  # log2 price
        preds_log.append(pred.detach().float().cpu().numpy())

    preds_log = np.concatenate(preds_log, axis=0) if len(preds_log) else np.array([])
    return preds_log, ds.ids, ds.missing_img_count

if TEST_CSV:
    print(f"🔮 Loading TEST_CSV: {TEST_CSV}")
    dft = build_test_df(TEST_CSV, ID_COL, TEXT_COL, IMG_COL, TEST_IMG_DIR)
    print(f"📝 Test rows: {len(dft)}")

    preds_log, test_ids, miss_count = infer_predictions(clip_model, price_head, processor, dft, TEXT_COL, IMG_COL, batch_size=max(32, BATCH_SIZE))
    if preds_log.size == 0:
        print("⚠️ No predictions generated for test.")
    else:
        pred_price = delog2(preds_log.reshape(-1))
        out_df = pd.DataFrame({ID_COL: test_ids, "price": pred_price})
        out_path = os.path.join(OUTPUT_DIR, "test_predictions.csv")
        out_df.to_csv(out_path, index=False)
        print(f"📤 Saved predictions to: {out_path}")
        print(f"⚠️ Test missing images encountered: {miss_count}")
else:
    print("ℹ️ TEST_CSV not set — skipping inference.")
# %%

🔮 Loading TEST_CSV: jl_fs/test.csv
📝 Test rows: 75000


You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


KeyboardInterrupt: 

In [16]:
# %% [markdown]
# --- Inference: load best checkpoint and predict on TEST_CSV ---

# %%
import os, json, math
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import torch
import torch.nn.functional as F
from torch.utils.data import DataLoader
from transformers import CLIPModel, AutoProcessor

# ---- Config / paths ----
TEST_CSV        = os.environ.get("TEST_CSV", "jl_fs/test.csv")   # must contain ID + text + image path
ID_COL          = os.environ.get("ID_COL", "sample_id")
TEXT_COL        = os.environ.get("TEXT_COL", "catalog_content")
IMG_COL         = os.environ.get("IMG_COL",  "image_path")

OUTPUT_DIR      = os.environ.get("OUTPUT_DIR", "price_clip_fulltrain")
CKPT_PATH       = os.environ.get("CKPT_PATH", os.path.join(OUTPUT_DIR, "full_clip.pt"))

BATCH_SIZE      = int(os.environ.get("INF_BATCH_SIZE", "64"))
MAX_LEN_ENV     = os.environ.get("MAX_LEN", None)  # if you want to override tokenizer max len
FP16            = os.environ.get("FP16", "true").lower() == "true"

assert os.path.exists(CKPT_PATH), f"Checkpoint not found at {CKPT_PATH}"
assert os.path.exists(TEST_CSV),  f"Test CSV not found at {TEST_CSV}"

device = "cuda" if torch.cuda.is_available() else "cpu"

# ---- Load checkpoint ----
ckpt = torch.load(CKPT_PATH, map_location="cpu")
model_id = ckpt.get("model_id", "openai/clip-vit-large-patch14")
cfg = ckpt.get("config", {})
projection_dim = cfg.get("projection_dim")
img_missing_policy = cfg.get("IMG_MISSING_POLICY", "zero")
max_len = int(cfg.get("MAX_LEN", 64)) if MAX_LEN_ENV is None else int(MAX_LEN_ENV)

print(f"📦 Loaded checkpoint from: {CKPT_PATH}")
print(f"🔤 MODEL_ID={model_id} | projection_dim={projection_dim} | IMG_MISSING_POLICY={img_missing_policy} | MAX_LEN={max_len}")

# ---- Recreate processor & models ----
processor = AutoProcessor.from_pretrained(model_id)
clip_model = CLIPModel.from_pretrained(model_id)
clip_model.load_state_dict(ckpt["clip_state"], strict=True)
clip_model.to(device).eval()

# Recreate and load regression head (same class as training cell)
price_head = ClipRegressionHead(embed_dim=projection_dim, dropout=0.0)
price_head.load_state_dict(ckpt["head_state"], strict=True)
price_head.to(device).eval()

# ---- Load test data ----
dft = pd.read_csv(TEST_CSV)
dft["image_path"] = dft["sample_id"].apply(lambda x : f"jl_fs/images/test/{x}.jpg")
for col, name in [(ID_COL, "ID_COL"), (TEXT_COL, "TEXT_COL"), (IMG_COL, "IMG_COL")]:
    if col not in dft.columns:
        raise ValueError(f"{name} '{col}' missing from test CSV. Columns={dft.columns.tolist()}")

# Basic clean
dft[TEXT_COL] = dft[TEXT_COL].fillna("").astype(str).str.strip()
dft[IMG_COL]  = dft[IMG_COL].fillna("").astype(str)

# Build dataset/dataloader with no targets
test_ds = ClipPriceDataset(
    df=dft[[ID_COL, TEXT_COL, IMG_COL]].copy(),
    text_col=TEXT_COL,
    img_col=IMG_COL,
    prices_log2=None,
    processor=processor,
    max_len=max_len,
    policy=img_missing_policy
)
collate = CollateClip(processor)

dl_te = DataLoader(
    test_ds, batch_size=BATCH_SIZE, shuffle=False,
    num_workers=2, pin_memory=True, collate_fn=collate
)

print(f"🖥 Device: {device}")
print(f"🧪 Test rows: {len(test_ds)} | Missing images encountered (during getitem): {test_ds.missing_img_count}")
print(f"🗑 Dropped due to policy=drop: {getattr(test_ds, 'dropped_missing', 0)}")

# ---- Inference loop ----
clip_model_dtype = next(clip_model.vision_model.parameters()).dtype
preds_log2 = []

with torch.no_grad():
    for batch in tqdm(dl_te, total = len(dl_te)):
        input_ids      = batch["input_ids"].to(device, non_blocking=True)
        attention_mask = batch["attention_mask"].to(device, non_blocking=True)
        img_missing    = batch["img_missing"].to(device)

        # Text features
        with torch.cuda.amp.autocast(enabled=FP16):
            txt_feat = clip_model.get_text_features(input_ids=input_ids, attention_mask=attention_mask)

            # Image features depending on policy
            do_vision = ("pixel_values" in batch) and (img_missing_policy != "text_only")
            if do_vision:
                pixel_values = batch["pixel_values"].to(device, dtype=clip_model_dtype, non_blocking=True)
                img_feat = clip_model.get_image_features(pixel_values=pixel_values)
                if img_missing.any():
                    img_feat = img_feat * (1.0 - img_missing.unsqueeze(1).float())
            else:
                img_feat = torch.zeros_like(txt_feat)

            # Normalize + fuse
            txt_n = F.normalize(txt_feat, dim=-1)
            img_n = F.normalize(img_feat, dim=-1)
            fused = torch.cat([img_n, txt_n], dim=-1)

            # Predict log2(price)
            pred_log = price_head(fused)
            preds_log2.append(pred_log.detach().float().cpu().numpy())

📦 Loaded checkpoint from: price_clip_fulltrain/full_clip.pt
🔤 MODEL_ID=openai/clip-vit-large-patch14 | projection_dim=768 | IMG_MISSING_POLICY=zero | MAX_LEN=64
🖥 Device: cuda
🧪 Test rows: 75000 | Missing images encountered (during getitem): 0
🗑 Dropped due to policy=drop: 0


  0%|          | 0/1172 [00:00<?, ?it/s]

You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
You're using a CLIPTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.
/tmp/ipykernel_97/3650335273.py:97: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(enabled=FP16):


In [20]:
# ---- Convert back to price (delog2) and save ----
if len(preds_log2):
    preds_log2 = np.concatenate(preds_log2, axis=0)
    preds_log2_delog = np.pow(2,preds_log2)
    preds_price = np.clip(preds_log2_delog, 0,10000)  # safe de-log clamp
else:
    preds_price = np.array([])

out = pd.DataFrame({
    ID_COL: dft[ID_COL].values[: len(preds_price)],
    "price": preds_price
})
pred_path = os.path.join(OUTPUT_DIR, "test_predictions_clip_big_full_v2.csv")
os.makedirs(OUTPUT_DIR, exist_ok=True)
out.to_csv(pred_path, index=False)

print(f"✅ Done. Wrote {len(out)} predictions to: {pred_path}")
print(f"   Missing images counted during dataset load: {test_ds.missing_img_count}")
print(f"   Dropped rows (policy=drop): {getattr(test_ds,'dropped_missing',0)}")

✅ Done. Wrote 75000 predictions to: price_clip_fulltrain/test_predictions_clip_big_full_v2.csv
   Missing images counted during dataset load: 0
   Dropped rows (policy=drop): 0
